In [1]:
import polars as pl

In [2]:
train_data = pl.read_parquet('otto-reduced/train.parquet')
test_data = pl.read_parquet('otto-reduced/test.parquet')
print(train_data)

shape: (3_340_193, 2)
┌─────────┬─────────────────────────────────┐
│ session ┆ events                          │
│ ---     ┆ ---                             │
│ i64     ┆ list[struct[3]]                 │
╞═════════╪═════════════════════════════════╡
│ 1185618 ┆ [{1020908,1659386928785,"click… │
│ 7575896 ┆ [{1440931,1660340322363,"click… │
│ 4293917 ┆ [{1679269,1659737781324,"click… │
│ 462188  ┆ [{69116,1659351401895,"clicks"… │
│ 1028060 ┆ [{1540638,1659380043176,"click… │
│ …       ┆ …                               │
│ 4365092 ┆ [{1361831,1659769672889,"click… │
│ 2210007 ┆ [{1508734,1659498037473,"click… │
│ 7272969 ┆ [{636101,1660301315572,"clicks… │
│ 7569078 ┆ [{320776,1660338958474,"clicks… │
│ 7712086 ┆ [{801774,1660384723701,"clicks… │
└─────────┴─────────────────────────────────┘


In [3]:
pl.Config.set_fmt_str_lengths(1000)
print(test_data)


shape: (6_699_195, 3)
┌──────────┬─────────┬────────────────────────────────────────────────────────────────────────┐
│ session  ┆ label   ┆ events                                                                 │
│ ---      ┆ ---     ┆ ---                                                                    │
│ i64      ┆ i64     ┆ list[struct[3]]                                                        │
╞══════════╪═════════╪════════════════════════════════════════════════════════════════════════╡
│ 12204380 ┆ 1157175 ┆ [{225122,1661502793261,"clicks"}, {277870,1661502813114,"clicks"}, …   │
│          ┆         ┆ {1074901,1661503932537,"clicks"}]                                      │
│ 11676028 ┆ 478901  ┆ [{1029424,1661302906256,"clicks"}, {1029424,1661302946617,"clicks"}, … │
│          ┆         ┆ {1293905,1661303179619,"clicks"}]                                      │
│ 11369945 ┆ 257383  ┆ [{479970,1661200978721,"clicks"}, {479970,1661200995782,"carts"}, …    │
│          ┆      

In [4]:
from polars.datatypes import Struct

class Recommender:
    def fit(self, data: pl.DataFrame) -> None:
        # fit the model to the training data
        pass
        
    def recommend(self, events: list[Struct]) -> list[int]:
        # return a list of k item ids
        pass

In [5]:
class BaselineRecommender(Recommender):
    def __init__(self):
        self.top_k = None
        
    def fit(self, data):
        data = data.explode( "events" ).unnest( "events" )
        print(data)
        self.top_k = data.group_by("aid").len().sort("len", descending=True).head(20)["aid"].to_list()
        
    def recommend(self, events):
        return self.top_k


bs_model = BaselineRecommender()
bs_model.fit(train_data)
print(bs_model.recommend([1, 2, 3]))

shape: (68_667_257, 4)
┌─────────┬─────────┬───────────────┬────────┐
│ session ┆ aid     ┆ ts            ┆ type   │
│ ---     ┆ ---     ┆ ---           ┆ ---    │
│ i64     ┆ i64     ┆ i64           ┆ str    │
╞═════════╪═════════╪═══════════════╪════════╡
│ 1185618 ┆ 1020908 ┆ 1659386928785 ┆ clicks │
│ 1185618 ┆ 1356523 ┆ 1659386944620 ┆ clicks │
│ 1185618 ┆ 130973  ┆ 1659387015519 ┆ clicks │
│ 1185618 ┆ 1392955 ┆ 1659387070030 ┆ clicks │
│ 1185618 ┆ 199292  ┆ 1659387078709 ┆ clicks │
│ …       ┆ …       ┆ …             ┆ …      │
│ 7712086 ┆ 1464627 ┆ 1660386545258 ┆ clicks │
│ 7712086 ┆ 1145198 ┆ 1660386575785 ┆ clicks │
│ 7712086 ┆ 1734182 ┆ 1660386592436 ┆ clicks │
│ 7712086 ┆ 627378  ┆ 1660386603044 ┆ clicks │
│ 7712086 ┆ null    ┆ 1661330073456 ┆ end    │
└─────────┴─────────┴───────────────┴────────┘
[None, 1460571, 29735, 108125, 231487, 1733943, 1502122, 756588, 832192, 986164, 670006, 1586171, 322370, 409620, 166037, 620545, 77440, 554660, 801774, 184976]


In [6]:
def evaluation(model, test_data):
    # evaluate the model on the test data
    right_predictions = 0
    i = 0
    print(len(test_data))
    for sequence in test_data.iter_rows():
        if i % 100000 == 0:
            print(i)
        recommendations = model.recommend(sequence[2])
        label = sequence[1]
        if label in recommendations:
            right_predictions += 1
        i += 1
        
    print(f"Accuracy: {right_predictions / len(test_data) * 100} %")

evaluation(bs_model, test_data)

6699195
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
Accuracy: 0.47601241641719644 %


In [14]:
smaller_train_data = train_data.head(1000)

In [18]:
from fastai.collab import CollabDataLoaders, collab_learner

class FastAIRecommender(Recommender):
    def __init__(self):
        print("init")
        recommender = None

    def fit(self, data: pl.DataFrame) -> None:
        data = data.explode( "events" ).unnest( "events" )
        print("exploded")
        data = data.to_pandas()
        print("to pandas")
        dls = CollabDataLoaders.from_df(data, bs=64)
        print("dls")
        learn = collab_learner(dls, y_range=(0, 1))
        print("learner")
        print(learn.fit_one_cycle(1))



In [19]:
FastAIRecommender().fit(smaller_train_data)

init
exploded
to pandas
dls
learner


None
